In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
import scipy as sc

In [34]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [35]:
Passenger_Id = test['PassengerId'].values
for_drop = ['PassengerId', 'Name']
train = train.drop(for_drop, axis=1)
test = test.drop(for_drop, axis=1)
all_data = pd.concat([train, test], axis=0)

In [36]:
to_factorize = [label for label in ['Sex', 'Ticket', 'Cabin', 'Embarked'] if not label in for_drop]
factorized_features = [pd.factorize(all_data[label].values, na_sentinel=0)[0] for label in to_factorize]
for i in range(len(to_factorize)):
    all_data[to_factorize[i]] = factorized_features[i]

imp = Imputer()
all_data['Age'] = imp.fit_transform(all_data['Age'].values.reshape(-1, 1))

ohe = OneHotEncoder(sparse=False)
dummy_features = ohe.fit_transform(all_data['Embarked'].values.reshape(-1, 1))
dummy_features = pd.DataFrame(dummy_features, columns=['Embarked = ' + str(x) for x in range(dummy_features.shape[1])])
all_data.index = dummy_features.index
all_data = pd.concat([all_data.drop(['Embarked'], axis=1), dummy_features], axis=1)

In [5]:
print all_data

            Age  Cabin      Fare  Parch  Pclass  Sex  SibSp  Survived  Ticket  \
0     22.000000      0    7.2500      0       3    0      1       0.0       0   
1     38.000000      0   71.2833      0       1    1      1       1.0       1   
2     26.000000      0    7.9250      0       3    1      0       1.0       2   
3     35.000000      1   53.1000      0       1    1      1       1.0       3   
4     35.000000      0    8.0500      0       3    0      0       0.0       4   
5     29.881138      0    8.4583      0       3    0      0       0.0       5   
6     54.000000      2   51.8625      0       1    0      0       0.0       6   
7      2.000000      0   21.0750      1       3    0      3       0.0       7   
8     27.000000      0   11.1333      2       3    1      0       1.0       8   
9     14.000000      0   30.0708      0       2    1      1       1.0       9   
10     4.000000      3   16.7000      1       3    1      1       1.0      10   
11    58.000000      4   26.

In [37]:
train, test = all_data[:len(train['Sex'])], all_data[len(train['Sex']):]
test = test.drop(['Survived'], axis=1)

In [38]:
X = train.drop(['Survived'], axis=1).as_matrix()
y = train['Survived'].values
X_test = test.as_matrix()
imp = Imputer(axis=1)
X_test = imp.fit_transform(X_test)

In [39]:
#clf = SVC()
def test_m(model):
    model.fit(X, y)
    ret = cross_val_score(model, X, y, cv=3)
    print ret, np.mean(ret)
    return ret, np.mean(ret)

In [59]:
test(LogisticRegression())
scores = []
for reg in np.arange(0, 1, 0.01)[1:]:
    print reg
    ret1, ret2 = test(LogisticRegression(C=reg))
    scores.append(ret2)
print np.arange(0, 1, 0.01)[1:][np.argmax(scores)]

[ 0.79461279  0.78451178  0.79461279] 0.791245791246
0.01
[ 0.71043771  0.71380471  0.76430976] 0.729517396184
0.02
[ 0.70707071  0.74747475  0.77777778] 0.744107744108
0.03
[ 0.72727273  0.75420875  0.7979798 ] 0.759820426487
0.04
[ 0.73737374  0.75084175  0.80808081] 0.765432098765
0.05
[ 0.75420875  0.76767677  0.81144781] 0.777777777778
0.06
[ 0.76430976  0.77777778  0.80808081] 0.783389450056
0.07
[ 0.76767677  0.78451178  0.80808081] 0.786756453423
0.08
[ 0.76767677  0.78451178  0.7979798 ] 0.783389450056
0.09
[ 0.76767677  0.78787879  0.79461279] 0.783389450056
0.1
[ 0.77777778  0.78114478  0.79124579] 0.783389450056
0.11
[ 0.77104377  0.77104377  0.7979798 ] 0.780022446689
0.12
[ 0.77441077  0.77104377  0.7979798 ] 0.781144781145
0.13
[ 0.78787879  0.77104377  0.7979798 ] 0.785634118967
0.14
[ 0.8013468   0.77104377  0.7979798 ] 0.79012345679
0.15
[ 0.8013468   0.77104377  0.7979798 ] 0.79012345679
0.16
[ 0.7979798   0.77104377  0.7979798 ] 0.789001122334
0.17
[ 0.8013468   0.7

In [70]:
scores = []
for weight in np.arange(0, 1, 0.01)[1:]:
    print weight
    ret1, ret2 = test(LogisticRegression(C=0.46, class_weight={0.:1, 1.:weight}))
    scores.append(ret2)
print np.arange(0, 1, 0.01)[1:][np.argmax(scores)]

0.01
[ 0.61616162  0.61616162  0.61616162] 0.616161616162
0.02
[ 0.61616162  0.61616162  0.61616162] 0.616161616162
0.03
[ 0.61616162  0.61279461  0.61616162] 0.615039281706
0.04
[ 0.61616162  0.62962963  0.61616162] 0.620650953984
0.05
[ 0.61616162  0.62962963  0.61616162] 0.620650953984
0.06
[ 0.62289562  0.63973064  0.61952862] 0.627384960718
0.07
[ 0.62626263  0.64646465  0.63636364] 0.636363636364
0.08
[ 0.63636364  0.65993266  0.64646465] 0.64758698092
0.09
[ 0.64309764  0.67003367  0.64646465] 0.653198653199
0.1
[ 0.64309764  0.67676768  0.65319865] 0.657687991021
0.11
[ 0.66666667  0.68013468  0.66329966] 0.670033670034
0.12
[ 0.67340067  0.68686869  0.68013468] 0.680134680135
0.13
[ 0.67340067  0.69023569  0.69023569] 0.684624017957
0.14
[ 0.68350168  0.69023569  0.6969697 ] 0.690235690236
0.15
[ 0.69360269  0.7003367   0.6969697 ] 0.69696969697
0.16
[ 0.69360269  0.7037037   0.71043771] 0.702581369248
0.17
[ 0.6969697   0.70707071  0.71380471] 0.705948372615
0.18
[ 0.6969697 

In [72]:
scores = []
for weight in np.arange(1, 100, 1)[1:]:
    print weight
    ret1, ret2 = test(LogisticRegression(C=0.46, class_weight={0.:1, 1.:weight}))
    scores.append(ret2)
print np.arange(1, 100, 1)[1:][np.argmax(scores)]

2
[ 0.76767677  0.75420875  0.78787879] 0.769921436588
3
[ 0.73737374  0.76094276  0.78451178] 0.760942760943
4
[ 0.70707071  0.77104377  0.73737374] 0.738496071829
5
[ 0.64309764  0.72390572  0.65993266] 0.675645342312
6
[ 0.5993266   0.65319865  0.57912458] 0.610549943883
7
[ 0.54208754  0.61279461  0.50841751] 0.5544332211
8
[ 0.52525253  0.56902357  0.49158249] 0.52861952862
9
[ 0.49494949  0.52188552  0.47138047] 0.496071829405
10
[ 0.49158249  0.4983165   0.44781145] 0.47923681257
11
[ 0.46801347  0.48148148  0.42087542] 0.456790123457
12
[ 0.45117845  0.47474747  0.41077441] 0.4455667789
13
[ 0.44444444  0.45117845  0.41414141] 0.436588103255
14
[ 0.43434343  0.44781145  0.40740741] 0.429854096521
15
[ 0.42760943  0.42760943  0.40740741] 0.420875420875
16
[ 0.42760943  0.41414141  0.40740741] 0.416386083053
17
[ 0.42760943  0.42087542  0.4006734 ] 0.416386083053
18
[ 0.42087542  0.41750842  0.39393939] 0.410774410774
19
[ 0.42424242  0.41414141  0.39393939] 0.410774410774
20
[ 0

In [40]:
test_m(LogisticRegression())
test_m(LogisticRegression(C=0.46))
test_m(LogisticRegression(C=0.46, class_weight={0.:1, 1.:0.71}))

[ 0.79461279  0.78451178  0.79461279] 0.791245791246
[ 0.8047138   0.78787879  0.8013468 ] 0.79797979798
[ 0.78451178  0.7979798   0.81818182] 0.800224466891


(array([ 0.78451178,  0.7979798 ,  0.81818182]), 0.80022446689113369)

In [41]:
alg = LogisticRegression(C=0.46, class_weight={0.:1, 1.:0.71})
alg.fit(X, y)
pred = alg.predict(X_test)

In [42]:
print pred

[ 0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.
  0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  1.  0.  1.  0.  1.  0.  1.  1.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  1.  1.  1.  0.  1.  1.  1.  0.
  0.  1.  1.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.
  0.  0.  1.  0.  0.  0.  1.  0.  1.  0.  1.  0.  0.  0.  1.  0.  0.  0.
  0.  1.  0.  1.  1.  1.  1.  0.  0.  1.  1.  1.  1.  0.  1.  0.  0.  1.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  1.  0.  1.  0.
  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.
  0.  1.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.
  0.  0.  1.  0.  1.  1.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  1.
  1.  0.  1.  0.  1.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  1.  1.  1.  1.  0.  0.  0.  0.  1

In [47]:
ret = np.array([Passenger_Id.astype(int), pred.astype(int)]).transpose()
pd.DataFrame(ret, columns=['PassengerId', 'Survived']).to_csv('predictions.csv', index=False)